In [1339]:
#importing the required directories

import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import glob
import os


%matplotlib inline
import sys

In [1340]:
#Helper function to check whether the given point lies within a rectangle or not,it is used in order to generate delaunay traingle
def rect_contains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[2] :
        return False
    elif point[1] > rect[3] :
        return False
    return True



In [1341]:
l=[];

#This function is used to generate the delaunay traingles for the selected control points

def draw_delaunay(img, subdiv, delaunay_color, dictionary) :
    
    triangleList = subdiv.getTriangleList();
    size = img.shape
    r = (0, 0, size[1], size[0])

    for t in triangleList :
        
        pt1 = (int(t[0]), int(t[1]))
        pt2 = (int(t[2]), int(t[3]))
        pt3 = (int(t[4]), int(t[5]))
        
        if rect_contains(r, pt1) and rect_contains(r, pt2) and rect_contains(r, pt3) :
            
            cv2.line(img, pt1, pt2, delaunay_color, 2)
            cv2.line(img, pt2, pt3, delaunay_color, 2)
            cv2.line(img, pt3, pt1, delaunay_color, 2)
            l.append((dictionary[pt1],dictionary[pt2],dictionary[pt3]))


In [1342]:
#Input image 1

img1=cv2.imread('trump.jpg')
img1_cpy=img1.copy()
# img1= cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
# print(img1.shape)
# cv2.imshow("trump",img1)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [1343]:
#Input image 2

img2=cv2.imread('clinton.jpg')
# img2= cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img2_cpy=img2.copy()
# print(img2.shape)
# cv2.imshow("clinton",img2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [1344]:
events = [i for i in dir(cv2) if 'EVENT' in i]
points1=[];

In [1345]:
#Mouse handler function with the defined task for LEFT-CLICK eevent,it helps us to select the desired control points on the image

def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
#         print(x,",",y)
        points1.append((int(x),int(y)))
        font = cv2.FONT_HERSHEY_SIMPLEX
        strXY ="."
        cv2.putText(img1_cpy, strXY, (x,y), font, 0.5, (255,255,0), 2)
        cv2.imshow("image", img1_cpy)

In [1346]:
#selecting control points in image 1 using mouse handler ans storing the co ordinates in a list 

points1.clear()
cv2.imshow("image", img1_cpy)
cv2.setMouseCallback("image", click_event)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [1347]:
events = [i for i in dir(cv2) if 'EVENT' in i]
points2=[]

In [1348]:
#Mouse handler function with the defined task for LEFT-CLICK eevent,it helps us to select the desired control points on the image

def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
#         print(x,",",y)
        points2.append((int(x),int(y)))
        font = cv2.FONT_HERSHEY_SIMPLEX
        strXY ="."
        cv2.putText(img2_cpy, strXY, (x,y), font, 0.5, (255,255,0), 2)
        cv2.imshow("image2", img2_cpy)

In [1349]:
#selecting control points in image 2 using mouse handler ans storing the co ordinates in a list

points2.clear()
cv2.imshow("image2", img2_cpy)
cv2.setMouseCallback("image2", click_event)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [1350]:
#Generating the Delaunany triangulation for the source and destination image using the control points selected
# print(len(points1))
# print(len(points2))

size = img1_pts.shape

#appending the corner points
points1.append((0,0))
points2.append((0,0))

points1.append((int(size[1]-1),int(size[0]-1)))
points2.append((int(size[1]-1),int(size[0]-1)))

points1.append((0,int(size[0]/2)))
points2.append((0,int(size[0]/2)))

points1.append((int(size[1]-1),int(size[0]/2)))
points2.append((int(size[1]-1),int(size[0]/2)))

points1.append((int(size[1]/2),0))
points2.append((int(size[1]/2),0))

points1.append((int(size[1]/2),int(size[0]-1)))
points2.append((int(size[1]/2),int(size[0]-1)))

points1.append((int(size[1]-1),0))
points2.append((int(size[1]-1),0))

points1.append((0,int(size[0]-1)))
points2.append((0,int(size[0]-1)))

# print(points1)

dictionary1={}
dictionary2={}

img1_tr=img1.copy()
img2_tr=img2.copy()

for  i in range(len(points1)):
    dictionary1[(points1[i][0],points1[i][1])]=i

for  i in range(len(points2)):
    dictionary2[(points2[i][0],points2[i][1])]=i
    
    
rect = (0, 0, size[1], size[0])
subdiv1 = cv2.Subdiv2D(rect)
subdiv2 = cv2.Subdiv2D(rect)
for p in points1 :
#     print(p)
    subdiv1.insert(p)
    
for p in points2 :
#     print(p)
    subdiv2.insert(p)

t_index1=[]
t_index2=[]
    
draw_delaunay( img1_tr, subdiv1, (255, 255, 255),dictionary1);
draw_delaunay( img2_tr, subdiv2, (255, 255, 255),dictionary2);

In [1351]:
cv2.imshow("trump_traingle",img1_tr)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite("Donald_Trump_triangulation.jpeg",img1_tr)

True

In [1352]:
cv2.imshow("clinton_traingle",img2_tr)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite("Hilaary_Clinton_traingulation.jpeg",img2_tr)

True

In [1363]:
def applyAffineTransform(src, srcTri, dstTri, size) :
    
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    
    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst

In [1364]:
def morphTriangle(img1, img2, img, t1, t2, t, alpha) :

    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))


    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []


    for i in range(0, 3):
        tRect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2Rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]

    size = (r[2], r[3])
    warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
    warpImage2 = applyAffineTransform(img2Rect, t2Rect, tRect, size)

    # Alpha blend rectangular patches
    imgRect = (1.0 - alpha) * warpImage1 + alpha * warpImage2

    # Copy triangular region of the rectangular patch to the output image
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + imgRect * mask

In [1365]:
alpha = np.array([0.0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95,1.00])

# for i in range(0, len(alpha)):
#     print(alpha[i])
    
img_array = []

In [1366]:
img_array.clear()

for k in range(0, len(alpha)):
    
    itr_points=[]
    #Obtaining control points  for the intermediate frame 
    for i in range(0, len(points1)):
        x = ( 1 - alpha[k] ) * points1[i][0] + alpha[k] * points2[i][0]
        y = ( 1 - alpha[k] ) * points1[i][1] + alpha[k] * points2[i][1]
        itr_points.append((x,y))    
    
    imgMorph = np.zeros(img1.shape, dtype = img1.dtype)

    for i in range(len(l)):

        x = int(l[i][0])
        y = int(l[i][1])
        z = int(l[i][2])

        t1 = [points1[x], points1[y], points1[z]]
        t2 = [points2[x], points2[y], points2[z]]
        t = [ itr_points[x], itr_points[y], itr_points[z] ]

        # Morph one triangle at a time.
        morphTriangle(img1, img2, imgMorph, t1, t2, t, alpha[k])

    img_array.append(imgMorph)

In [1367]:
for i in range(0, len(img_array)):
    
    label="Morph_"+ str(i+1)+".jpeg"
    cv2.imwrite(label,img_array[i])

In [1368]:
video_name = 'morphing.avi' 
height,width,layers=img_array[0].shape

video = cv2.VideoWriter(video_name, 0, 1, (width, height))  

for i in range(0, len(img_array)):
    video.write(img_array[i])  
       
cv2.destroyAllWindows()  
video.release()